In [1]:
from rdflib import Graph

In [7]:
from pyfuseki import FusekiQuery
from api.src.schemas.settings import Settings

settings = Settings()

authorityQuery = FusekiQuery(f'{settings.url}:3030', 'authority')

def GraphExist(uri):  
    
    token = uri.split("/")[-1]

    query = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
            PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
            PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            SELECT ?uri ?label WHERE {{ graph ?g {{
            ?sub bf:identifiedBy ?identifiedBy .
    ?identifiedBy rdf:value "{token}" .
    ?uri madsrdf:authoritativeLabel ?label .   
                }} }} """
    responseQuery = authorityQuery.run_sparql(query)
    bindings = responseQuery.convert()['results']['bindings']
    if len(bindings) > 0:
        return bindings[0]
    else: None


def GetInternalUri(authority, graph, metadata, obj):

    query = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
            SELECT * WHERE  {{
              <{authority}> madsrdf:has{metadata}Authority ?value .
              ?value madsrdf:authoritativeLabel ?label
              }}"""
    response = graph.query(query)
    if len(response.bindings) > 0:
        external = list()
        internal = list()
        for i in response.bindings:
           url = i.get('value')
           exist = GraphExist(url)
 
           if exist:
                uri = {
                   "value": exist['uri']['value'],
                   "base": "bk",
                   "label": {
                       "value": exist['label']['value'],
                       "lang": "por" } }
                internal.append(uri)
           else:
               uri = {
                   "value": url,
                   "base": "loc",
                   "label": {
                       "value": i.get('label').value,
                       "lang": i.get('label').language } }
               external.append(uri)
        if len(internal) > 0:
            obj[f'has{metadata}Authority'] = internal
        if len(external) > 0:
            obj[f'has{metadata}ExternalAuthority'] = external
    return obj

In [10]:
def GetExternalUri(authority, graph, metadata, obj):

    query = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
            SELECT * WHERE  {{
              <{authority}> madsrdf:has{metadata}Authority ?value .
              ?value madsrdf:authoritativeLabel ?label
              }}"""
    response = graph.query(query)
    if len(response.bindings) > 0:
        external = list()
        for i in response.bindings:
            url = i.get('value')
            base = url.split("//")[1].split("/")[0]
            uri = {
                   "value": url,
                   "base": base,
                   "label": {
                       "value": i.get('label').value,
                       "lang": i.get('label').language } }
            external.append(uri)
        return external
    else:
        return False

In [41]:
from api.src.schemas.authorities.authority import AdminMetadata, Uri, Variant
from api.src.schemas.authorities.authority import Element

from pydantic import BaseModel
from typing import Optional

class Subject(BaseModel):
    type: str 
    adminMetadata: AdminMetadata 
    elementList: list[Element]
    note: Optional[str] = None
    hasReciprocalAuthority: Optional[list[Uri]] = None
    hasReciprocalExternalAuthority: Optional[list[Uri]] = None
    hasBroaderAuthority: Optional[list[Uri]] = None
    hasBroaderExternalAuthority: Optional[list[Uri]] = None
    hasNarrowerAuthority: Optional[list[Uri]] = None
    hasNarrowerExternalAuthority: Optional[list[Uri]] = None
    hasCloseExternalAuthority: Optional[list[Uri]] = None
    hasExactExternalAuthority: Optional[list[Uri]] = None
    hasVariant: Optional[list[Variant]] = None
    subjectOf: Optional[list[Uri]] = None
    isMemberOfMADSCollection: str

In [14]:
from api.src.routes.translate.makeTranslate import MakeTranslate

def GetElementList(authority, graph, obj):

    translator = MakeTranslate(
            source_language='en',
            target_language='pt',
            timeout=10
        )

    qElementList = f"""PREFIX identifiers: <http://id.loc.gov/vocabulary/identifiers/>
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
  SELECT ?elementValue ?type WHERE  {{
    <{authority}> madsrdf:elementList ?o .
    ?o rdf:rest* ?node .
    ?node rdf:first ?e .
    ?e madsrdf:elementValue ?elementValue .
    ?e rdf:type ?type
    }}"""
    r = graph.query(qElementList)
    elementList = list()
    
    for i in r.bindings:
        value = translator.translate(i.get('elementValue').value)
        element = {
            "type": i.get('type').split("#")[1],
          "elementValue": {
            "value":  value.capitalize(),
            "lang": 'pt'
          }
        }
        elementList.append(element)
    obj['elementList'] = elementList
    return obj

In [17]:
from api.src.routes.translate.makeTranslate import MakeTranslate

def GetVariant(authority, graph, obj):

    qVariant = f"""PREFIX identifiers: <http://id.loc.gov/vocabulary/identifiers/>
                PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
                SELECT ?typeVariant ?typeElement ?elementValue WHERE  {{
                <{authority}> madsrdf:hasVariant ?variant .
                ?variant rdf:type ?typeVariant .
                ?variant madsrdf:elementList ?elementList .
                ?elementList rdf:rest* ?node .
                  ?node rdf:first ?e .
                  ?e madsrdf:elementValue ?elementValue .
                ?e rdf:type ?typeElement .
                FILTER ( ?typeVariant != madsrdf:Variant )
                }}"""
    r = graph.query(qVariant)
    if len(r.bindings) > 0:
        translator = MakeTranslate(
            source_language='en',
            target_language='pt',
            timeout=10
        )
        variants = list()
        for i in r.bindings:
            value = translator.translate(i.get('elementValue').value)
            variant = {
          'type': i.get('typeVariant').split("#")[1],
          'elementList': [{
              'type': i.get('typeElement').split("#")[1],
              'elementValue': {
                  'value': value.capitalize(),
                  'lang': 'pt'
              }
          }]
      }
            variants.append(variant)
        obj['hasVariant'] = variants
    return obj

In [20]:
from api.src.routes.translate.makeTranslate import MakeTranslate

def Note(graph, authority, obj):

    qNote = f"""PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
  SELECT ?note WHERE {{ 
      <{authority}> madsrdf:note ?note .
       }}"""
    
    translator = MakeTranslate(
            source_language='en',
            target_language='pt',
            timeout=10
        )
    
    r = graph.query(qNote)
    if len(r.bindings) > 0:
        value = translator.translate(r.bindings[0].get('note').value)
        
        obj['note'] = value
    return obj

In [24]:
# from api.src.function.loc.ElementList import GetElementList
# from api.src.function.loc.Variant import GetVariant
from api.src.function.loc.getType import GetType

def ParserSubject(graph, authority): 

  # Type
  tipo = GetType(graph, authority)
  
  # adminMetadata
  adminMetadata = {
      "assigner": "http://id.loc.gov/vocabulary/organizations/dlc",
      "identifiedBy": [ {
         "type": "Lccn",
          "assigner": "http://id.loc.gov/vocabulary/organizations/dlc",
          "value": authority.split('/')[-1]        
      }]      
  }
  
  obj = {
     "type": tipo,
      "adminMetadata": adminMetadata,
      "isMemberOfMADSCollection": f'http://bibliokeia.com/authorities/{tipo}/'
  }
  
  # ElementList
  obj = GetElementList(authority, graph, obj)

  # Note 
  obj = Note(graph, authority, obj)

  # hasVariant
  obj = GetVariant(authority, graph, obj)

  # URIS
  # hasReciprocalAuthority
  obj = GetInternalUri(authority, graph, "Reciprocal", obj)

  # hasBroaderAuthority
  obj = GetInternalUri(authority, graph, "Broader", obj)
  
  # Narrower Terms
  obj = GetInternalUri(authority, graph, "Narrower", obj)
  
#   hasNarrower
  hasNarrower = GetExternalUri(authority, graph, "NarrowerExternal", obj)
  if hasNarrower:
    if obj.get('hasNarrowerExternalAuthority'): 
       obj['hasNarrowerExternalAuthority'] = obj['hasNarrowerExternalAuthority']+hasNarrower
    else:
       obj['hasNarrowerExternalAuthority'] = hasNarrower
       
  
  # ExactExternal
  exactExternal = GetExternalUri(authority, graph, "ExactExternal", obj)
  if exactExternal:
    obj['hasExactExternalAuthority'] = exactExternal
  
  # CloseExternal
  closeExternal = GetExternalUri(authority, graph, "CloseExternal", obj)
  if closeExternal:
    obj['hasCloseExternalAuthority'] = closeExternal

  # response = Subject(**obj)

  return obj



In [22]:
uri = 'http://id.loc.gov/authorities/subjects/sh88005635'
graph = Graph()
graph.parse(f'{uri}.rdf') 

<Graph identifier=Ncd1d9ae2ee7a4208a62bd0e6819dcd7a (<class 'rdflib.graph.Graph'>)>

In [ ]:
subject = ParserSubject(graph, uri)
subject

In [43]:
request = Subject(**subject)
request.model_dump()

c:\BiblioKeia\venv\Lib\site-packages\pydantic\main.py:292: UserWarning: Pydantic serializer warnings:
  Expected `datetime` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


{'type': 'Topic',
 'adminMetadata': {'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc',
  'descriptionModifier': 'http://id.loc.gov/vocabulary/organizations/brmninpa',
  'creationDate': datetime.date(2023, 7, 17),
  'descriptionLanguage': 'http://id.loc.gov/vocabulary/languages/por',
  'generationProcess': 'BiblioKeia v.1',
  'generationDate': '2023-07-17T14:07:51',
  'identifiedBy': [{'type': 'Lccn',
    'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc',
    'value': 'sh88005635'}],
  'status': {'value': 'mstatus:new', 'label': 'novo'}},
 'elementList': [{'type': 'TopicElement',
   'elementValue': {'value': 'Ecofisiologia vegetal', 'lang': 'pt'}}],
 'note': None,
 'hasReciprocalAuthority': None,
 'hasReciprocalExternalAuthority': None,
 'hasBroaderAuthority': None,
 'hasBroaderExternalAuthority': [{'value': 'http://id.loc.gov/authorities/subjects/sh85102796',
   'label': {'value': 'Plant physiology', 'lang': 'en'},
   'base': 'loc'},
  {'value': 'http://id.loc.go

In [46]:
def MakeLabel(elementList):
    labels = [i.elementValue.value for i in elementList]
    label = ", ".join(labels)
    
    return label

def MakeDocSubject(request, id):
    nMeta = ['type', 'adminMetadata', 'elementList', 'hasVariant', 'note', 'isMemberOfMADSCollection']
    doc = {
            'id': id,
            'type': request.type,
            "creationDate": request.adminMetadata.creationDate.strftime('%Y-%m-%d'), 
            "label": f'{MakeLabel(request.elementList)}' ,
            "isMemberOfMADSCollection": request.isMemberOfMADSCollection
        }
    if request.note:
        doc['note'] = request.note
    if request.hasVariant:
        variants = list()
        for i in request.hasVariant:
            label = [j.elementValue.value for j in i.elementList]
            label = "--".join(label)
            variants.append(label)
        doc['variant'] = variants
    for k, v in request:
        if v and k not in nMeta:
                uris = list()
                for i in v:
                        uri = {
                                'id': f"{id}/{k}#{i.value.split('/')[-1]}",
                                'uri': i.value, 
                                'label': i.label.value, 
                                'lang': i.label.lang,
                                'base': i.base
                                }
                        uris.append(uri)
                doc[f'{k}'] = uris
    return doc

In [49]:
doc = MakeDocSubject(request, 'bka-1') 
doc

{'id': 'bka-1',
 'type': 'Topic',
 'creationDate': '2023-07-17',
 'label': 'Ecofisiologia vegetal',
 'isMemberOfMADSCollection': 'http://bibliokeia.com/authorities/Topic/',
 'variant': ['Fisiologia vegetal ambiental',
  'Ecologia vegetal fisiológica',
  'Ecologia fisiológica vegetal'],
 'hasBroaderExternalAuthority': [{'id': 'bka-1/hasBroaderExternalAuthority#sh85102796',
   'uri': 'http://id.loc.gov/authorities/subjects/sh85102796',
   'label': 'Plant physiology',
   'lang': 'en',
   'base': 'loc'},
  {'id': 'bka-1/hasBroaderExternalAuthority#sh91006274',
   'uri': 'http://id.loc.gov/authorities/subjects/sh91006274',
   'label': 'Ecophysiology',
   'lang': 'en',
   'base': 'loc'},
  {'id': 'bka-1/hasBroaderExternalAuthority#sh85015985',
   'uri': 'http://id.loc.gov/authorities/subjects/sh85015985',
   'label': 'Plant ecology',
   'lang': 'en',
   'base': 'loc'}],
 'hasNarrowerExternalAuthority': [{'id': 'bka-1/hasNarrowerExternalAuthority#sh85102753',
   'uri': 'http://id.loc.gov/auth

In [52]:
from pysolr import Solr

In [53]:
solr = Solr(f'{settings.url}:8983/solr/authority/', timeout=10)

In [56]:
responseSolr = solr.add([doc], commit=True)
responseSolr

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":279}}\n'

In [57]:
uri = 'http://id.loc.gov/authorities/names/n79125202'
graph = Graph()
graph.parse(f'{uri}.rdf')

<Graph identifier=Nac11203cdb22493e952ebff3a3d808c5 (<class 'rdflib.graph.Graph'>)>

In [58]:
from api.src.function.loc.Uri import GetUri
from api.src.function.loc.agents.Affiliation import GetAffiliation
from api.src.function.loc.agents.BirthPlace import GetBirthPlace
from api.src.function.loc.agents.Date import GetDate
from api.src.function.loc.agents.FullerName import GetFullerName
from api.src.function.loc.agents.Occuption import GetOccuption
from api.src.function.loc.agents.fieldOfActivity import GetFieldOfActivity


def ParserAgents(graph, authority):
    # Type
    tipo = GetType(graph, authority)

    # adminMetadata
    adminMetadata = {
      "assigner": "http://id.loc.gov/vocabulary/organizations/dlc", 
      "identifiedBy": [ {
         "type": "Lccn",
          "assigner": "http://id.loc.gov/vocabulary/organizations/dlc",
          "value": authority.split('/')[-1]        
      }]}
    
    obj = {
     "type": tipo,
      "adminMetadata": adminMetadata,
      "isMemberOfMADSCollection": f'http://bibliokeia.com/authorities/{tipo}/'}
    
    # ElementList
    obj = GetElementList(graph, authority, obj) 
    
    # fullerName
    obj = GetFullerName(graph, authority, obj)

    # hasCloseExternalAuthority
    obj = GetUri(authority, graph, "hasCloseExternalAuthority", obj)

    # hasExactExternalAuthority
    obj = GetUri(authority, graph, "hasExactExternalAuthority", obj)

    # Variant
    obj = GetVariant(authority, graph, obj)

    # RWO
    token = authority.split("/")[-1]
    rwo = f'http://id.loc.gov/rwo/agents/{token}'
    # BirthDate
    obj = GetDate(rwo, 'birthDate', graph, obj)
    # deathDate
    obj = GetDate(rwo, 'deathDate', graph, obj)
    # BirthPlace
    obj = GetBirthPlace(rwo, graph, obj)
    # Affiliation
    obj = GetAffiliation(rwo, graph, obj)
    # Occuptions
    obj = GetOccuption(rwo, graph, obj)
    obj = GetFieldOfActivity(rwo, graph, obj)

    return obj

    # response = Agents(**obj)
    
    # return response

In [59]:
response = ParserAgents(graph, uri)
response

AttributeError: 'str' object has no attribute 'query'